Segmenting and Clustering Neighborhoods in Toronto

In [1]:
!pip install beautifulsoup4
from bs4 import BeautifulSoup 

import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!pip install geopy
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

# Matplotlib and plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means 
from sklearn.cluster import KMeans

!pip install folium
import folium 

!pip install geocoder
import geocoder # longitude and latitude

     |████████████████████████████████| 92kB 19.8MB/s eta 0:00:01
     |████████████████████████████████| 102kB 23.6MB/s ta 0:00:01


1. Table of Neighborhoods in Toronto

In [2]:
# Webpage
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url).text

# Extract only table
soup = BeautifulSoup(page, 'html.parser')
table = soup.find('table', class_='sortable')

# Get the values from the Wikipedia table and store into a dataframe
row = [] # initialize row list

for tr in table.find_all('tr'):                                          
    if tr.find_all('th') == []:                                           
        row.append([td.get_text(strip=True) for td in tr.find_all('td')]) 
        
# Assign columns names and turn row into a dataframe of neighborhoods
column_names = ['Postcode', 'Borough', 'Neighborhood']

neighborhoods_raw = pd.DataFrame(row, columns=column_names) # create a raw table of neighborhoods
neighborhoods_raw.head(10)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [3]:
# Rows containing 'Not assigned' boroughs are dropped.
drop_index = neighborhoods_raw[neighborhoods_raw['Borough'] == 'Not assigned'].index 
neighborhoods = neighborhoods_raw.drop(drop_index, axis=0)                          
neighborhoods.reset_index(drop=True, inplace=True)                                   


In [4]:
# Assign borough names to neighborhoods when neighborhood is 'Not assigned'
nh_na = neighborhoods[neighborhoods['Neighborhood'] == 'Not assigned'].index # index of those rows
neighborhoods.iloc[nh_na, 2] = neighborhoods['Borough'][nh_na]

In [5]:
# Neighborhoods with same postcode merged into single cell
neighborhoods = neighborhoods.groupby(['Postcode', 'Borough'], as_index=False).agg(lambda x: ', '.join(x))
neighborhoods

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [6]:
neighborhoods.shape

(103, 3)

2. Longitude and Latitude of Neighborhoods

In [7]:
# Initialize varialbes
lat = []
lng = []
lat_lng_coords = None

# Get postcodes from neighborhoods table
postal_code = neighborhoods['Postcode']

# Store latitude and longitude values in lat and lng
for pc in postal_code:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(pc))
    lat_lng_coords = g.latlng
    lat.append(lat_lng_coords[0])
    lng.append(lat_lng_coords[1])

In [8]:
# Add lists with latitude and longitude values to neighborhood dataframe
nh_complete = neighborhoods
nh_complete['Latitude'] = lat
nh_complete['Longitude'] = lng

In [9]:
nh_complete

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726245,-79.263670
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713133,-79.285055
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.723575,-79.234976
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696665,-79.260163


3. Explore and cluster Neighborhoods

In [10]:
# New dataframe only with original city included
toronto = nh_complete[nh_complete['Borough'].str.find('Toronto') != -1].reset_index(drop=True)
toronto.shape

(38, 5)

In [11]:
# Get the latitude and longitude of Toronto
g = geocoder.arcgis('Toronto, Ontario')
lat_tor = g.latlng[0]
lng_tor = g.latlng[1]

# Create a map of Toronto
map_toronto = folium.Map(location=[lat_tor, lng_tor], zoom_start=11)

# Add markers to map
for lat, lng, bor, postcode in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Postcode']):
    label = '{}, {}'.format(postcode, bor)        # popup labels with postcode and borough
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [14]:
## Define Foursquare Credentials and Version
CLIENT_ID = 'PI2SENV35OWCXGAQJEO31230N1MRIWUG5WEACRUN3TGVR0YR' # Foursquare ID
CLIENT_SECRET = 'ND5VAKZRU2XVWO05P2LVKHQI3CA0YY1EMA2CCCL0X0SZI0TT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PI2SENV35OWCXGAQJEO31230N1MRIWUG5WEACRUN3TGVR0YR
CLIENT_SECRET:ND5VAKZRU2XVWO05P2LVKHQI3CA0YY1EMA2CCCL0X0SZI0TT


In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(name,
                             lat,
                             lng,
                             v['venue']['name'],
                             v['venue']['location']['lat'],
                             v['venue']['location']['lng'],
                             v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
# Limit the number of venues to 100
LIMIT = 100

# Get the venues for each postcode
toronto_venues = getNearbyVenues(names=toronto['Postcode'],
                                 latitudes=toronto['Latitude'],
                                 longitudes=toronto['Longitude'],
                                 radius=700) # limit radius to 700 meters

In [17]:
# Check the venues dataframe
print(toronto_venues.shape)
toronto_venues.head(10)

(2401, 7)


,Postcode,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676845,-79.295225,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,M4E,43.676845,-79.295225,Glen Manor Ravine,43.676821,-79.293942,Trail
2,M4E,43.676845,-79.295225,The Beech Tree,43.680493,-79.288846,Gastropub
3,M4E,43.676845,-79.295225,Beaches Bake Shop,43.680363,-79.289692,Bakery
4,M4E,43.676845,-79.295225,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
5,M4E,43.676845,-79.295225,Mastermind Toys,43.671453,-79.293971,Toy / Game Store
6,M4E,43.676845,-79.295225,Beacher Cafe,43.671938,-79.291238,Breakfast Spot
7,M4E,43.676845,-79.295225,Veloute Bistro,43.672267,-79.289584,French Restaurant
8,M4E,43.676845,-79.295225,Castro's Lounge,43.671104,-79.295107,Bar
9,M4E,43.676845,-79.295225,Sanna's Farmacia,43.670929,-79.295969,Juice Bar


In [18]:
# Check the number of venues for each postcode
num_venues = pd.DataFrame()
num_venues['Postcode'] = toronto_venues.groupby('Postcode').count().reset_index()['Postcode']
num_venues['Venue Count'] = toronto_venues.groupby('Postcode').count().reset_index()['Venue']
num_venues

,Postcode,Venue Count
0,M4E,27
1,M4K,31
2,M4L,33
3,M4M,76
4,M4N,8
5,M4P,23
6,M4R,36
7,M4S,39
8,M4T,7
9,M4V,22


Calculating Mean Frequency of occurences for each category for each Neighborhood

In [19]:
# One hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# Add Postcode column
toronto_onehot['Postcode'] = toronto_venues['Postcode'] 

# Move postcode column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Postcode').mean().reset_index()
toronto_grouped.head(10)

,Postcode,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Lawyer,Library,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Tram Station,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,M4E,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.074074,0.000000,0.074074,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.037037,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.074074,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.037037,0.000000,0.0,0.000000,0.0,0.0,0.0

Table with characteristics for each neighborhood

In [20]:
# Function that returns the sorted categories
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]                                       
    row_categories_sorted = row_categories.sort_values(ascending=False) 
    
    return row_categories_sorted.index.values[0:num_top_venues]         

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind])) # 1st, 2nd, 3rd
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Postcode'] = toronto_grouped['Postcode']

# Each row of the new dataframe is filled with the sorted categories of each row of toronto_goruped dataframe
for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Café,Pub,Japanese Restaurant,Bar,Bakery,Tea Room,Toy / Game Store,Juice Bar,French Restaurant,Neighborhood
1,M4K,Greek Restaurant,Ice Cream Shop,Café,Yoga Studio,Brewery,Bubble Tea Shop,Fast Food Restaurant,Business Service,Scenic Lookout,Chinese Restaurant
2,M4L,Park,Coffee Shop,Brewery,Fast Food Restaurant,Sandwich Place,Café,Bakery,Burrito Place,Board Shop,Steakhouse
3,M4M,Coffee Shop,Café,Bar,Bakery,Diner,Italian Restaurant,Pizza Place,Sandwich Place,Fast Food Restaurant,Arts & Crafts Store
4,M4N,Park,Gym / Fitness Center,Restaurant,Café,Coffee Shop,Lawyer,Bookstore,Bus Line,Event Space,Ethiopian Restaurant


Cluster Neighborhood into 3 Clusters and create Map

In [22]:
# Set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Postcode', 1)

# Run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 1], dtype=int32)

In [23]:
# Add clustering labels
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto

# Merge toronto_grouped with toronto to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('Postcode'), on='Postcode')
toronto_merged[['Cluster Labels']] = toronto_merged[['Cluster Labels']].astype('int')

toronto_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676845,-79.295225,1,Café,Pub,Japanese Restaurant,Bar,Bakery,Tea Room,Toy / Game Store,Juice Bar,French Restaurant,Neighborhood
1,M4K,East Toronto,"The Danforth West, Riverdale",43.683262,-79.355120,1,Greek Restaurant,Ice Cream Shop,Café,Yoga Studio,Brewery,Bubble Tea Shop,Fast Food Restaurant,Business Service,Scenic Lookout,Chinese Restaurant
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.667965,-79.314673,1,Park,Coffee Shop,Brewery,Fast Food Restaurant,Sandwich Place,Café,Bakery,Burrito Place,Board Shop,Steakhouse
3,M4M,East Toronto,Studio District,43.662766,-79.334830,1,Coffee Shop,Café,Bar,Bakery,Diner,Italian Restaurant,Pizza Place,Sandwich Place,Fast Food Restaurant,Arts & Crafts Store
4,M4N,Central Toronto,Lawrence Park,43.728160,-79.387085,1,Park,Gym / Fitness Center,Restaurant,Café,Coffee Shop,Lawyer,Bookstore,Bus Line,Event Space,Ethiopian Restaurant


In [24]:
# Create map
map_clusters = folium.Map(location=[lat_tor, lng_tor], zoom_start=11)

# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i * x) ** 2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, pc, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postcode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(pc) + ', ' + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon],
                        radius=5,
                        popup=label,
                        color=rainbow[cluster-1],
                        fill=True,
                        fill_color=rainbow[cluster-1],
                        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters